# Trabajo N.º 2 
## Modelamiento de patrones sísmicos en Chile a través de regresión, clasificación y clustering



###  Abstract

Este trabajo analiza la **actividad sísmica en Chile (2012–2025)** con el objetivo de aplicar técnicas de **regresión, clasificación y agrupamiento** para responder hipótesis de Inteligencia de Negocios (BI).

Se utiliza un dataset de sismos del **Centro Sismológico Nacional (CSN)**, que contiene variables como magnitud, profundidad, latitud, longitud y fecha.  
El análisis comienza con la limpieza y estandarización de los datos, seguido por una **exploración descriptiva y correlacional** que permite identificar las variables más influyentes sobre la magnitud de los eventos.

Posteriormente, se desarrolla una **regresión** para explicar la magnitud según factores geográficos, una **clasificación** para distinguir eventos de alta magnitud, y un **agrupamiento (clustering)** para descubrir patrones espaciales en la actividad sísmica nacional.  
Las métricas utilizadas incluyen **MAE, RMSE y R²** (para regresión), **accuracy, precisión, recall y F1-score** (para clasificación), y **silhouette e inercia** (para clustering).

Finalmente, se discuten los resultados en términos de su utilidad para la toma de decisiones estratégicas en BI, identificando posibles zonas de riesgo y oportunidades para optimizar la gestión de alertas sísmicas.


###  Objetivos

**Objetivo general:**  
Analizar y modelar la actividad sísmica en Chile para identificar patrones relevantes que apoyen decisiones de Inteligencia de Negocios mediante técnicas de regresión, clasificación y agrupamiento.

**Objetivos específicos:**
1. Desarrollar una **exploración de datos (EDA)** con correlaciones, histogramas y visualizaciones temporales para comprender las relaciones entre magnitud, profundidad y ubicación.  
2. Implementar un **modelo de regresión** que estime la magnitud en función de variables geográficas y de profundidad.  
3. Aplicar un **modelo de clasificación** para identificar eventos de **alta magnitud**, evaluando su rendimiento con métricas de exactitud y F1-score.  
4. Ejecutar un **modelo de clustering (K-Means y DBSCAN)** para segmentar los sismos según sus características espaciales y magnitud, evaluando la calidad de los grupos formados.


###  Hipótesis

- **H1 (Regresión):** La **profundidad** y la **posición geográfica (latitud y longitud)** influyen significativamente en la **magnitud** de los sismos ocurridos en Chile.  
- **H2 (Clasificación):** Es posible **predecir correctamente** si un sismo será de **alta magnitud** usando variables de ubicación y profundidad con un rendimiento superior al azar (F1 > 0.6).  
- **H3 (Clustering):** Existen **agrupamientos naturales** de sismos que reflejan diferencias geográficas (norte, centro y sur), identificables mediante métricas de calidad como el **coeficiente de silhouette**.


In [1]:
import pandas as pd, sklearn, matplotlib
print("✅ Entorno OK:", pd.__version__)


✅ Entorno OK: 2.3.3


In [4]:
# =============================================================================
# CARGA Y PREPARACIÓN INICIAL DE DATOS
# =============================================================================
import os
import glob
import pandas as pd

print("Cargando dataset...")

# -------------------------------------------------------------------------
# Configuración general: nombre esperado y estructura recomendada
# Estructura esperada del ZIP:
# BI_T2_CardenasNicolas/
# ├── BI_T2_CardenasNicolas.ipynb
# ├── data/
# │   └── seismic_data.csv
# -------------------------------------------------------------------------
NOMBRE_ARCHIVO = "seismic_data.csv"
RUTAS_CANDIDATAS = [
    os.path.join("data", NOMBRE_ARCHIVO),      # 1) ./data/
    NOMBRE_ARCHIVO,                            # 2) carpeta actual
    os.path.join("..", "data", NOMBRE_ARCHIVO) # 3) ../data/
]

# Buscar el archivo en esas rutas
archivo_encontrado = next((p for p in RUTAS_CANDIDATAS if os.path.exists(p)), None)

# Búsqueda recursiva adicional (por si cambió el nombre de carpeta)
if not archivo_encontrado:
    resultados = glob.glob(f"**/{NOMBRE_ARCHIVO}", recursive=True)
    if resultados:
        archivo_encontrado = resultados[0]

# Validación final
if not archivo_encontrado:
    print(f"\n❌ ERROR: No se encuentra el archivo '{NOMBRE_ARCHIVO}'.")
    print("Asegúrate de incluirlo dentro de la carpeta 'data/' al comprimir el proyecto.")
    print(f"Directorio actual: {os.getcwd()}")
    print("Archivos en esta carpeta:", os.listdir("."))
    raise FileNotFoundError(f"No se encontró el archivo '{NOMBRE_ARCHIVO}'")

# -------------------------------------------------------------------------
# Cargar el CSV con tolerancia a codificación y separador 
# -------------------------------------------------------------------------
def cargar_csv_robusto(ruta):
    """Intenta leer el CSV usando diferentes configuraciones para evitar errores."""
    for args in [
        {"sep": ",", "encoding": "utf-8"},
        {"sep": ";", "encoding": "utf-8"},
        {"sep": ",", "encoding": "latin-1"},
        {"sep": ";", "encoding": "latin-1"},
    ]:
        try:
            df = pd.read_csv(ruta, **args)
            if len(df.columns) > 1:
                return df
        except Exception:
            continue
    raise ValueError(f"No se pudo leer el archivo CSV '{ruta}' con los encodings probados.")

df = cargar_csv_robusto(archivo_encontrado)

print(f"✓ Dataset cargado exitosamente desde: {archivo_encontrado}\n")

# -------------------------------------------------------------------------
# Información básica del dataset 
# -------------------------------------------------------------------------
print("=" * 70)
print("INFORMACIÓN BÁSICA DEL DATASET PRUEBA")
print("=" * 70)
print(f"Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
print("Columnas:", list(df.columns))

# Intento de detectar columna de fecha
fecha_cols = [c for c in df.columns if any(k in str(c).lower() for k in ["fecha", "date", "time"])]
col_fecha = None
for c in fecha_cols:
    try:
        aux = pd.to_datetime(df[c], errors="coerce")
        if aux.notna().any():
            col_fecha = c
            break
    except Exception:
        continue

if col_fecha:
    serie_fecha = pd.to_datetime(df[col_fecha], errors="coerce")
    print(f"Período: {serie_fecha.min()} a {serie_fecha.max()}  (columna: '{col_fecha}')")
else:
    print("Período: no identificado (no se detectó columna de fecha legible)")

print("\nPrimeras filas del dataset:")
display(df.head(10))


Cargando dataset...
✓ Dataset cargado exitosamente desde: data\seismic_data.csv

INFORMACIÓN BÁSICA DEL DATASET PRUEBA
Dimensiones: 4,018 filas × 5 columnas
Columnas: ['Date(UTC)', 'Latitude', 'Longitude', 'Depth', 'Magnitude']
Período: 2012-03-03 11:01:47 a 2025-05-26 03:50:27  (columna: 'Date(UTC)')

Primeras filas del dataset:


,Date(UTC),Latitude,Longitude,Depth,Magnitude
0,2025-05-26 03:50:27,-19.63,-69.49,97,5.6
1,2025-05-13 00:47:58,-51.25,-72.28,28,5.1
2,2025-05-05 09:46:48,-29.49,-71.84,48,5.0
3,2025-05-05 02:17:48,-31.89,-70.88,88,5.1
4,2025-05-02 15:23:49,-27.52,-72.48,30,5.8
5,2025-05-02 12:58:32,-56.88,-68.06,10,7.5
6,2025-04-24 03:20:08,-30.76,-71.90,31,5.4
7,2025-04-18 00:10:18,-23.47,-68.22,139,6.0
8,2025-04-08 22:30:45,-21.05,-68.56,126,5.2
9,2025-04-08 10:28:43,-17.99,-69.93,143,4.8
